# Setup and imports

In [222]:
import pandas as pd
import numpy as np
import random
from collections import Counter
from __future__ import print_function
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from numpy import linalg as LA
import json
%matplotlib inline
import matplotlib.pyplot as plt


import nltk
import re
from nltk import pos_tag, word_tokenize
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/benstevens/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/benstevens/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [141]:
from html.parser import HTMLParser

class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ''.join(self.fed)

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

# Tea reviews

In [142]:
tea_df = pd.read_csv("scraper/clean_data.csv")
tea_ids = set(tea_df['id'])

In [482]:
def get_flavors(tId):
    return list(tea_df[tea_df['id'] == tId]['flavors'])[0]

In [28]:
tea_df

,id,name,brand,reviewCount,ratingValue,teaType,ingredients,flavors,soldIn,caffeine,certification,wantIt,ownIt,imageUrl,url
0,65303,2015 Last Thoughts,White 2 Tea,16.0,89.0,Pu-erh Tea,Not available,"Floral, Rainforest, Honeysuckle, Orchids, Pepp...",Not available,Not available,Not available,51.0,20.0,https://media.steepster.com/api/file/kL8dCqJrT...,https://steepster.com/teas/white-2-tea/65303-2...
1,43095,Golden Orchid,Whispering Pines Tea Company,155.0,89.0,Black Tea,"Fujian Xiao Zhong, Vanilla Bean, Yunnan Dian Hong","Dark Chocolate, Chocolate, Dates, Malt, Orchid...",Loose Leaf,High,Not available,315.0,236.0,https://media.steepster.com/api/file/exT0OHWdT...,https://steepster.com/teas/whispering-pines-te...
2,78801,The Silver Moonlight,Chai & Mighty,6.0,89.0,White Tea,White Tea Leaves,"Honey, Lemon, Nuts, Flowers, Lemongrass",Loose Leaf,Low,"Fair Trade, Organic, Vegan",3.0,4.0,https://media.steepster.com/api/file/cmNMAbwSn...,https://steepster.com/teas/chai-and-mighty/788...
3,48058,Cocoa Amore,Whispering Pines Tea Company,85.0,89.0,Black Tea,"Chinese Black Tea, Tahitian Vanilla","Cocoa, Dark Chocolate, Malt, Vanilla, Apple, A...",Loose Leaf,High,Not available,227.0,156.0,https://media.steepster.com/api/file/0vAf5X4Si...,https://steepster.com/teas/whispering-pines-te...
4,32444,Laoshan Bilochun Green,Verdant Tea,19.0,89.0,Green Tea,Not available,"Sweet, Mineral, Nutty, Roasted, Salty, Butter,...",Not available,Not available,Not available,11.0,21.0,https://media.steepster.com/api/file/F0AJQO5DT...,https://steepster.com/teas/verdant-tea/32444-l...
5,22442,Milk Oolong Tea,Mandala Tea,116.0,89.0,Oolong Tea,Not available,"Apricot, Creamy, Mango, Milk, Smooth, Stonefru...",Loose Leaf,Not available,Not available,314.0,127.0,https://media.steepster.com/api/file/Hat4DLjkT...,https://steepster.com/teas/mandala-tea/22442-m...
6,78800,The Queen of Greens,Chai & Mighty,12.0,89.0,Green Tea,Green Tea Leaves,"Broccoli, Cut grass, Nutty, Olives, Freshly Cu...","Bulk, Loose Leaf",Low,"Fair Trade, Organic, Vegan",8.0,2.0,https://media.steepster.com/api/file/jVMEypgaQ...,https://steepster.com/teas/chai-and-mighty/788...
7,20060,Hand Picked Tieguanyin Spring Oolong (2011),Verdant Tea,80.0,89.0,Oolong Tea,Oolong Tea,"Citrus Zest, Cream, Toasted Rice, Tobacco, Sug...",Loose Leaf,Medium,Not available,140.0,142.0,https://media.steepster.com/api/file/84cnuVtQM...,https://steepster.com/teas/verdant-tea/20060-h...
8,28779,Premium Taiwanese Assam,Butiki Teas,127.0,89.0,Black Tea,Black Tea Leaves,"Tea, Chocolate, Cream, Cocoa, Fruity, Malt, Br...",Loose Leaf,Not available,Not available,256.0,190.0,https://media.steepster.com/api/file/eXa2B8fyR...,https://steepster.com/teas/butiki-teas/28779-p...
9,41785,Imperial Gold Needle Yunnan Black Tea,Yunnan Sourcing,25.0,89.0,Black Tea,Not available,"Chocolate, Cocoa, Malt, Melon, Plums, Sweet Po...",Loose Leaf,Not available,Not available,40.0,33.0,https://media.steepster.com/api/file/grfipnEHQ...,https://steepster.com/teas/yunnan-sourcing/417...


In [143]:
unfiltered_tea_revs = pd.read_csv("reviews.csv")
unfiltered_tea_revs['description'] = unfiltered_tea_revs['description'].apply(strip_tags)
tea_revs = unfiltered_tea_revs[unfiltered_tea_revs.id.isin(tea_ids)]

In [144]:
sorted_revs = tea_revs.sort_values(by=["id", "likes"], ascending=[True, False])

In [145]:
top_sorted_revs = sorted_revs.groupby('id').head(3)

In [147]:
top_sorted_revs.to_csv("clean_reviews.csv", encoding='utf-8', index=False)

In [148]:
def review_lookup(tea_id):
    return tea_revs[tea_revs['id'] == tea_id].sort_values(by='likes',ascending=False)

In [149]:
review_lookup(2)

,id,review_id,note_id,author,ratingValue,description,likes
185838,2,172340,242752,CHAroma,70,"I thought I’d rated this before?…hmm, I guess...",14
185120,2,84753,135590,Sil,65,I can get behind this one in summer as a cold...,14
185110,2,16073,29389,takgoti,9,Jillian Lena sent me this tea. Here is my rev...,13
185121,2,61346,100887,Mercuryhime,20,After reading the…um…opinionated reviews on t...,13
185119,2,13360,24984,LENA,25,Smells like a cleaner Did I eat a vitamin? Ta...,13
185725,2,257663,353728,Indigobloom,69,My last tea of the day. Taken with dinner at ...,10
185130,2,125041,114833,Will Work For Tea,Not available,Last night I developed a wet cough – yeah gro...,9
185132,2,40001,68301,Michelle Butler Hallett,2,(Should my dear friends who gave me this tea ...,9
185301,2,24167,42990,JonTea,44,Looks like the haters delight… However I see ...,8
185832,2,183331,258482,Booker,90,Beautiful orange scent right out of the bag. ...,8


In [185]:
def reviews_by_id(tea_id):
    #return list(zip(review_lookup(tea_id)['description'].tolist(), review_lookup(tea_id)['ratingValue'].tolist()))
    return review_lookup(tea_id)['description'].tolist()


In [187]:
def top_reviews_by_id(tea_id):
    return reviews_by_id(tea_id)[:3]

In [188]:
def build_top_reviews():
    review_dict = {}
    for tea_id in tea_ids:
        review_dict[tea_id] = top_reviews_by_id(tea_id)
    return review_dict

In [189]:
reviews_dict = build_top_reviews()

# Tokenization and descriptors

In [64]:
tea_descriptions = tea_revs['description']
descriptions = []
i = 0
for desc in tea_descriptions:
    descriptions.append(desc)

In [65]:
def get_adj(tea_id):
    return list(set([t[0] for t in pos_tag(word_tokenize(" ".join(reviews_dict[tea_id]))) if t[1] == "JJ"]))

In [66]:
def get_adjectives(words):
    return set([t[0] for t in pos_tag(words) if t[1] == "JJ"])

In [274]:
def build_vectorizer(max_features, stop_words, max_df=0.6, min_df=10, norm='l2'):
    """Returns a TfidfVectorizer object
    
    Params: {max_features: Integer,
             max_df: Float,
             min_df: Float,
             norm: String,
             stop_words: String}
    Returns: TfidfVectorizer
    """
    # YOUR CODE HERE
    v = TfidfVectorizer(stop_words=stop_words, max_df=max_df, min_df=min_df, max_features=max_features, norm=norm, use_idf=True, smooth_idf=True)
    return v
    
    
    raise NotImplementedError()

In [275]:
n_feats = 20000
tfidf_vec = build_vectorizer(n_feats, "english", min_df=70)
doc_by_vocab = tfidf_vec.fit_transform(d for d in descriptions).toarray()
index_to_vocab = {i:v for i, v in enumerate(tfidf_vec.get_feature_names())}
vocab = {v for i, v in enumerate(tfidf_vec.get_feature_names())}

/Users/benstevens/Desktop/Spring_2018/INFO4300/venv/lib/python3.5/site-packages/sklearn/feature_extraction/text.py:1039: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [439]:
adjectives = get_adjectives(vocab)
remove = ["hot", "cold", "good", "bad", "don", "pretty", 
          "nice", "little", "steep", "steeps", "sweet", 
          "leaf", "second", "know", "online", "brew", 
          "sugar", "erh", "loose", "new", "sure", "sent",
         "huge", "able", "great", "wasn", "mug", "kid", 
          "finish", "write", "type", "compliment", "face", 
          "kitchen", "hint", "equivalent", "tasting",
         "imo", "desk", "im", "previous", "small", "old",
         "festival", "journey", "interested"]
for r in remove:
    adjectives.remove(r)

In [486]:
def get_top_adj(id):
    toks = []
    for rev in reviews_by_id(id):
        toks.extend(word_tokenize(rev))
    return [t.capitalize() for (t,n) in Counter([w for w in toks if w in adjectives]).most_common(5)]
def get_top_adj_n(id, n):
    toks = []
    for rev in reviews_by_id(id):
        toks.extend(word_tokenize(rev))
    return [t.capitalize() for (t,n) in Counter([w for w in toks if w in adjectives]).most_common(n)]

In [487]:
get_top_adj(10)

['Jasmine', 'Black', 'Strong', 'Sweetness', 'Bold']

In [488]:
def format_string(l):
    s = ""
    for w in l:
        s += w + ", "
    return s[:-2]

# Formatting Data

In [489]:
ids = list(reviews_dict.keys())

In [490]:
review1 = [("" if len(reviews_dict[tid]) < 1 else reviews_dict[tid][0]) for tid in ids]

In [491]:
review2 = [("" if len(reviews_dict[tid]) < 2 else reviews_dict[tid][1]) for tid in ids]

In [492]:
review3 = [("" if len(reviews_dict[tid]) < 3 else reviews_dict[tid][2]) for tid in ids]

In [493]:
features_flavors = [get_flavors(tid) + ", " + format_string(get_top_adj_n(tid, 10)) for tid in ids]

In [476]:
features = [format_string(get_top_adj(tid)) for tid in ids]

In [477]:
features

['Sour, Lemon, Strong, Herbal, Favorite',
 'Black, Sachet, Strong, Favorite, Organic',
 'Jasmine, Green, Strong, Light, Floral',
 'Strong, Sour, Herbal, Favorite, Sweetener',
 'Malt, Floral, Subtle, Golden, Apparent',
 'Jasmine, Black, Strong, Sweetness, Bold',
 'Floral, Jasmine, Green, Oolong, Deep',
 'Floral, Oolong, Creamy, Green, Strong',
 'Oolong, Floral, Soft, Grass, Green',
 'Floral, Green, Strong, Creamy, Oolong',
 'Green, Light, Grass, Dry, Delicate',
 'Green, Oolong, Grass, Floral, Delicious',
 'Chamomile, Lemon, Strong, Herbal, Favorite',
 'Black, Oolong, Floral, Light, Dry',
 'Black, Malt, Light, Sweetness, Soft',
 'Flush, Local, Black, Organic, Light',
 'Black, Strong, Sachet, Soy, Sensitive',
 'Soy, Odd, Appropriate, Strong, Wish',
 'Herbal, Tropical, Medicinal, Tisane, Lemon',
 'Red, Light, Big, Sweetener, Free',
 'Black, Strong, Dry, Rich, Amazing',
 'Bring, Black, Excessive, Sensitive, Super',
 'Delicious, Black, Free, Strong, Odd',
 'White, Light, Needle, Delicate, Su

In [494]:
review_data = {'id': ids, 'review1': review1, 'review2': review2, 'review3': review3, 'features': features, 'features_flavors': features_flavors}
review_data_df = pd.DataFrame(data=review_data)

In [495]:
joined = tea_df.set_index('id').join(review_data_df.set_index('id'))


In [496]:
review_data_df.to_csv("features_data.csv", encoding='utf-8', index=False)